# Automação Web e Busca de Informações com Python

#### Desafio: 

Caso hipotético :trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver(pesquisar sobre webdriver do Chrome no google, fazer o download e colocar o arquivo na pasta do anaconda)

In [15]:
# intalando a lib do selenium
!pip install selenium

In [16]:
#importando as bibliotecas 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [17]:
#Importante, antes de começar o projeto em si, escrevermos um passo a passo.

#passo 1: cotação de dolar (acessar o navegador, buscar por cotação do dolar no google e retornar o resultado)

# abrir o navegador
navegador = webdriver.Chrome()

#entrar no google
navegador.get("https://www.google.com.br/")

#pesquisar cotação do dolar no google
#escreve na posição extata informada pelo xpath(caminho do site)
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
#aperta enter
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#copia o resultado na posição exata do xpath
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# passo 2: cotação do euro

#entrar no google
navegador.get("https://www.google.com.br/")

#pesquisar cotação do dolar no google 
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# passo 3: cotação do ouro

#nesse caso entrei no site melhor cambio
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(',', '.') #aqui dei um replace para substituir a virgula por ponto, ja que o python trabalha com ponto em casa decimais 
print(cotacao_ouro)

navegador.quit() #fecha o navegador

 



5.2443
5.3549
292.78


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [18]:
# passo 4: atualizar a base de dados
import pandas as pd #importando pandas

ds = pd.read_excel('Produtos.xlsx') #dei o nome para base de dados de 'ds'(dataset)

display(ds) # display para visualizar a base de dados

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [19]:
# passo 5: recalcular os preços

#atualizar as cotações

ds.loc[ds['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar) # encontrando a linha 'Dolar' na coluna 'Moeda' e substituindo os valores na coluna cotação pelo valor atual da cotação e convertendo em número.
ds.loc[ds['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro) #o mesmo para linha que estiver escrito 'Euro'
ds.loc[ds['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)# o mesmo para linha que estiver escrito 'Ouro'

#nos proximos passo tempos que fazer algumas contas para atualizar o resto da tabela com os valores da cotação atual:
#preço de compra = preço original * cotação 

ds['Preço de Compra'] = ds["Preço Original"] * ds['Cotação']

#preço de venda = preço de compra * margem 
ds['Preço de Venda'] = ds['Preço de Compra'] * ds['Margem']

display(ds)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2443,5244.247557,1.40,7341.946580
1,Carro Renault,4500.00,Euro,5.3549,24097.050000,2.00,48194.100000
2,Notebook Dell,899.99,Dólar,5.2443,4719.817557,1.70,8023.689847
3,IPhone,799.00,Dólar,5.2443,4190.195700,1.70,7123.332690
4,Carro Fiat,3000.00,Euro,5.3549,16064.700000,1.90,30522.930000
5,Celular Xiaomi,480.48,Dólar,5.2443,2519.781264,2.00,5039.562528
6,Joia 20g,20.00,Ouro,292.7800,5855.600000,1.15,6733.940000


### Agora vamos exportar a nova base de preços atualizada

In [20]:
#convertendo e criando um novo arquivo, nesse passo pode ser convertido para  extensão que desejar, no meu caso escolhi excel.
# passo 6 : exportar a base de dados
ds.to_excel("Produtos Novo.xlsx", index = False)